In [1]:
from network import CNetwork

In [2]:
import pandas as pd
import numpy as np

In [3]:
from utils import verifyDir

In [4]:
name_dataset = "dataset_1"
dataset_path = "datasetsv2/"
length_cut = 50000
random_flag = True
remove_stop_words = True
only_stop_words = False
remove_puntuaction = True
top_words = "top_50"
measures = ["dgr_n"]#, "btw", "cc", "sp", "sp_std", "accs_h2", "accs_h3"]

In [5]:
auxiliar_path = 'auxiliar_folder/' + name_dataset   + '/'
verifyDir(auxiliar_path)

In [6]:
df_train = pd.read_csv(dataset_path + "df_train_" + name_dataset + "_" + str(length_cut) + "_" + str(random_flag) + ".csv")

In [7]:
df_val = pd.read_csv(dataset_path + "df_val_" + name_dataset + "_" + str(length_cut) + "_" + str(random_flag) + ".csv")

In [8]:
df_test = pd.read_csv(dataset_path + "df_test_" + name_dataset + "_" + str(length_cut) + "_" + str(random_flag) + ".csv")

In [9]:
df_train.head(5)

,label,text,book
0,Bram Stoker,"Produced by eagkw , Robert Cicconetti and the ...",The Mystery Of The Sea
1,Allan Poe,Contents THE DEVIL IN THE BELFRY LIONIZING X-I...,The Works Of Edgar Allan Poe4
2,Allan Poe,Contents THE PURLOINED LETTER THE THOUSAND-AND...,The Works Of Edgar Allan Poe2
3,Hector Hugh,Transcribed from the 1914 John Lane edition by...,When William Came
4,Charles Dickens,"Also please be aware if spell-checking , that ...",Barnaby Rudge


In [10]:
from utils.text_processing import get_word_index, get_sequences, get_common_words, get_top_words, pre_process_text

In [11]:
df_train['text'] = df_train['text'].apply(lambda t: pre_process_text(t, remove_stop_words=remove_stop_words, only_stop_words=only_stop_words, remove_puntuaction_flag=remove_puntuaction))

In [12]:
df_test['text'] = df_test['text'].apply(lambda t: pre_process_text(t, remove_stop_words=remove_stop_words, only_stop_words=only_stop_words, remove_puntuaction_flag=remove_puntuaction))

In [13]:
df_val['text'] = df_val['text'].apply(lambda t: pre_process_text(t, remove_stop_words=remove_stop_words, only_stop_words=only_stop_words, remove_puntuaction_flag=remove_puntuaction))

In [14]:
df_val.head()

,label,text,book
0,Charles Darwin,"[Many, parts, admirably, adapted, giving, insi...",The Structure And Distribution Of Coral Reefs
1,Allan Poe,"[Produced, Ron, Swanson, NARRATIVE, ARTHUR, GO...",The Narrative Of Arthur Gordon Pym
2,Pelham Grenville,"[Produced, Suzanne, L, Shell, Charles, Franks,...",The Man With Two Left Feet
3,Arthur Conan Doyle,"[Produced, Judith, Boss, HTML, version, Al, Ha...",The Lost World
4,Jane Austen,"[Huntingdon, exclaimed, greatness, match, uncl...",Mansfield Park


In [15]:
def get_local_features(sequences, word_features, measures):
    network_features = pd.DataFrame()
    for text in sequences:
        obj = CNetwork(text, model=None, index_word=None, percentages=None, path=auxiliar_path)
        network = obj.create_network()
        local_measure = obj.get_network_measures(network, word_features, measures)
        a_series = pd.Series(local_measure)
        network_features = network_features.append(a_series, ignore_index=True)
        #network_features = np.vstack([network_features, global_measure]) if len(network_features) > 0 else global_measure
    return network_features

In [16]:
texts = df_train["text"]
word_index, index_word = get_word_index(texts)
sequences = get_sequences(texts, word_index)
word_features = get_top_words(top_words, sequences)
print(word_features)

{'4466': 0, '577': 1, '3807': 2, '492': 3, '3021': 4, '2033': 5, '3969': 6, '4500': 7, '4045': 8, '3946': 9, '423': 10, '4858': 11, '2377': 12, '1357': 13, '340': 14, '2124': 15, '646': 16, '43': 17, '4341': 18, '4115': 19, '304': 20, '841': 21, '1132': 22, '161': 23, '1257': 24, '809': 25, '100': 26, '2442': 27, '1058': 28, '291': 29, '1792': 30, '4306': 31, '1094': 32, '4328': 33, '3914': 34, '4334': 35, '2179': 36, '3576': 37, '23': 38, '1893': 39, '1393': 40, '24': 41, '3027': 42, '1596': 43, '510': 44, '4828': 45, '2396': 46, '1759': 47, '3102': 48, '3828': 49}


In [17]:
X_train = get_local_features(sequences, word_features, measures)

Nodes: 5383 - Edges: 18203
MEASURE [26.37142857 19.9122807  17.54166667 15.44642857 22.30232558 20.88888889
 12.42647059 18.85       13.62254902 15.71296296 17.58695652 16.06382979
 12.9047619  29.60416667 15.40566038  5.08333333 18.81034483 17.77160494
 13.93814433 15.27272727  9.25       18.80851064 15.87850467 21.38461538
 16.2        13.75       18.65625    11.0625     12.44444444 19.05
 11.95833333 40.26923077 20.34285714 14.90540541 21.44642857 19.30487805
 10.13636364  8.33928571 17.05963303 18.38235294 17.25       23.08108108
 17.4        13.43085106 14.225      16.63636364 26.2173913  16.09659091
 15.25806452 19.85344828         nan         nan         nan         nan
         nan  5.08333333         nan 11.0625             nan 16.2
         nan 14.         40.26923077         nan         nan         nan
 18.38235294 12.44444444         nan 18.85       12.9047619  12.70454545
 26.2173913  29.60416667         nan         nan         nan 14.89285714
 18.81034483 18.225          

/home/mailaucq/Documents/book_classification/venv/lib/python3.8/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


Nodes: 7262 - Edges: 20157
MEASURE [28.5        21.11111111 18.03846154 15.36196319 19.625      16.87
  9.71052632 26.08536585 14.21724138 23.85714286 22.03521127 18.17857143
 20.43902439 37.48148148 24.175      35.81818182 44.16666667 14.63917526
 20.69354839 24.69444444 10.875      16.54255319 24.17021277 14.51587302
 35.95454545  9.98611111 25.265625   28.22580645  3.75       15.31578947
 16.1875     24.72413793 26.58333333 10.53539823 25.87837838 12.64705882
 15.2734375  23.69230769 12.25555556 23.28125    36.33333333 13.13333333
 40.9137931  18.84883721 14.35416667 36.97727273 25.54347826 21.20987654
 13.36111111 25.70588235         nan         nan         nan  7.3125
         nan 26.58333333         nan 17.90625            nan         nan
 35.81818182         nan         nan 18.17857143         nan         nan
 12.64705882 24.84722222  9.71052632 28.5                nan 36.46590909
 25.54347826 29.26041667         nan 20.86538462 37.48148148         nan
 32.81896552         nan 2

Nodes: 6023 - Edges: 18136
MEASURE [17.175      11.90566038 14.13333333 12.93287037 29.13043478 18.32894737
 14.78571429 25.97826087 15.21256039 18.51785714 16.57954545 14.44927536
 26.7125      9.60344828 18.70833333  7.81818182  7.1        15.49065421
 14.34046053 32.59375    28.06666667 11.05172414 20.62068966 17.97619048
 13.4        14.96078431 20.43076923 16.86206897 25.1875     18.07692308
 24.67647059 20.30555556 22.25735294 15.60824742 27.28205128 10.70833333
 12.26315789 11.22222222 14.83050847 18.8        20.26388889 16.33088235
 18.55769231 17.54320988 23.55714286 17.81578947 26.7037037  19.65053763
 16.86666667 22.31481481         nan         nan         nan         nan
  7.1                nan         nan 25.1875             nan 13.4
  7.81818182         nan         nan         nan 21.1        32.59375
 24.67647059 11.22222222 17.81578947 17.175      14.78571429         nan
 27.55434783 10.70833333         nan 18.55769231 26.7037037          nan
 15.6954023  16.86666667  

Nodes: 6309 - Edges: 17741
MEASURE [ 45.47058824  23.08108108   6.91666667  19.57037037  56.55357143
  26.703125    31.46551724  46.57142857  21.40186916  20.14117647
  25.43956044  30.796875    38.70833333  47.2125      23.74568966
  38.91666667  79.875       22.8266129   27.72631579  62.52173913
  26.73529412  17.99494949  15.16666667  20.86842105   8.3
  27.84090909  30.23484848   9.35714286  40.4375      27.70967742
 208.          70.          36.515625    19.07926829  37.86363636
  48.72222222  14.         105.6875      27.75735294  53.94
  42.85294118  32.75757576  68.25        24.39        26.98245614
  42.17647059  32.03846154  36.59821429  14.42857143  32.2745098
          nan 208.                  nan          nan          nan
          nan          nan  92.78125     70.           8.3
          nan  22.91666667  32.03846154  14.42857143  15.16666667
  40.4375      39.30882353          nan          nan  68.25
          nan          nan  38.26086957          nan  53.94
        

         nan         nan         nan]
Len features: 50
[11.5        29.47297297 18.95833333 14.54       35.62       35.48113208
 28.40625    54.42105263 24.27419355 25.79166667 28.74561404 27.57142857
 40.77941176 25.51724138 33.68131868 24.75       51.63461538 20.01342282
 26.05508475 21.26190476 10.42857143 10.83571429 34.73076923 22.57916667
 38.17647059 25.29268293 28.1369863   7.02857143 33.33333333 32.20833333
 18.90625    24.4516129  24.796875   17.50763359 22.05128205 18.38235294
 20.88135593 31.34375    12.48       20.16666667 43.03846154 31.85869565
 54.89655172 31.34146341 26.69565217 31.47368421 39.125      24.17123288
 16.75       23.        ]
Nodes: 5744 - Edges: 18971
MEASURE [44.025      37.93055556  1.5        15.50773196 38.79807692 25.95614035
 28.76086957 46.51724138 19.00342466 16.95754717 20.6344086  40.20731707
 31.46153846 36.34210526 29.671875   21.9375     43.41666667 18.05555556
 27.125      45.2        13.88888889 29.5625     25.96610169 33.71774194
 11.0833

Nodes: 7587 - Edges: 21731
MEASURE [32.85714286 16.14864865  6.55       11.41780822 16.05555556 24.23076923
 19.6        21.8877551   8.875      15.09090909 15.98076923  7.
 15.97169811 12.16666667 27.7        29.625       7.75       14.09574468
  8.86666667 27.0625     11.9375     17.41428571 11.          7.73076923
  5.375      14.33695652 22.         16.84745763  7.4        14.7037037
 20.45833333 13.5        17.16666667  9.40140845 11.5        33.5
 15.28571429  5.5        11.54       20.86363636 22.225       8.125
 22.95454545 11.38461538 10.5862069  12.38636364 10.56       13.89534884
  8.91666667 13.61538462         nan  8.66666667         nan  7.9375
         nan 33.5                nan 14.79166667         nan 19.03333333
 17.22727273 19.66666667         nan 32.85714286  8.86666667         nan
         nan 13.52777778         nan 22.225              nan 16.625
         nan 14.66666667 15.08       15.98076923         nan 15.28571429
 10.5862069   6.55               nan         n

Nodes: 5718 - Edges: 18772
MEASURE [22.22727273 31.24074074 40.79166667 17.27678571 27.44565217 30.94186047
 12.36363636 14.70833333 17.31683168 19.42063492 22.5        17.88194444
 38.90540541 25.75       24.52293578 16.91666667 12.03571429 19.38383838
 23.59545455 31.22727273 25.         14.94594595 23.19491525 20.31404959
 17.55       15.6122449  23.63559322 21.14285714 19.5        16.40909091
  6.75       25.8125     25.09589041 25.07142857 24.45       19.38461538
 12.54411765 22.97115385 23.71875    25.1969697  26.72916667 23.28640777
 37.75       21.13235294 23.96590909 25.38571429 27.42857143 20.65865385
 20.74242424 20.22142857         nan  6.75               nan         nan
         nan 18.20833333         nan         nan         nan 17.55
         nan 40.79166667         nan 24.89285714         nan         nan
         nan         nan         nan         nan 27.42857143 26.72727273
         nan 22.41666667         nan         nan         nan         nan
         nan         n

Nodes: 5817 - Edges: 18665
MEASURE [29.68181818 23.06451613  4.66666667 14.29113924 27.64864865 20.32653061
  8.5        20.8        17.64285714 27.09090909 15.11702128 21.28461538
 17.35483871 18.75       22.17948718 27.65625    29.11111111 17.6
 15.40037594 14.80555556 18.40740741 15.60869565 39.71153846 21.49090909
 18.33333333 11.26315789 22.98734177 20.02941176 21.80769231 16.
 18.25       15.64       20.12820513 15.78625954 18.3        25.39855072
 17.96        8.90909091 15.28021978 15.24418605 24.09375    25.70491803
 47.33333333 23.17045455 25.60810811 12.22727273 14.41666667 17.25555556
 19.81818182 18.16666667         nan         nan         nan         nan
         nan  4.66666667         nan         nan 29.11111111 18.25
         nan 47.33333333 21.80769231         nan 20.8        27.65625
 20.02941176 14.80555556         nan         nan 13.41666667 24.75
         nan         nan 15.64       29.23076923 18.40740741         nan
         nan         nan 20.20967742 24.09375 

Nodes: 4108 - Edges: 17230
MEASURE [17.08333333 22.08988764 10.9375     16.26814516 39.69354839 27.51612903
 16.24137931 38.58571429 29.9537037  43.55882353 37.72222222 25.78571429
 21.92105263 20.16666667 36.91791045 30.47727273 32.79166667 21.8422619
 26.21212121 28.6        28.30555556 21.13513514 25.66363636 22.67307692
 32.24074074 19.45       23.51694915 23.82051282 31.         23.16891892
  5.40909091 29.63636364 22.29069767 18.58606557 22.25657895 35.125
 28.24358974 18.31034483 25.09322034 44.375      23.98958333 27.35555556
 47.39473684 21.62820513 35.         54.725      17.47222222 22.87096774
 21.70454545 21.80434783         nan         nan         nan         nan
         nan         nan         nan 10.9375             nan 31.
  5.40909091 32.79166667         nan         nan         nan 35.125
 43.55882353 27.83333333 34.65789474 49.55       35.         26.09090909
         nan 18.625              nan         nan 32.24074074         nan
 17.27586207         nan 39.6935483

In [18]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,26.371429,19.912281,17.541667,15.446429,22.302326,20.888889,12.426471,18.850000,13.622549,15.712963,...,17.250000,23.081081,17.400000,13.430851,14.225000,16.636364,26.217391,16.096591,15.258065,19.853448
1,6.545455,18.981132,12.416667,16.365942,31.458333,18.318182,10.122222,29.078125,15.629630,18.239437,...,17.155556,17.851351,25.907407,4.537037,18.960784,33.962500,5.666667,20.419753,8.944444,4.911765
2,16.950000,14.000000,14.250000,11.873563,21.619048,21.657895,19.014286,13.678571,11.823864,23.453125,...,10.482143,24.185185,26.413043,9.661290,16.988889,17.250000,10.166667,9.115385,12.021277,11.691489
3,21.000000,18.788889,25.392857,11.813253,20.260870,15.903061,9.217391,14.574074,13.876238,15.119048,...,10.975000,25.056604,18.531250,11.974684,13.170000,24.870370,9.250000,14.847059,10.375000,19.939655
4,37.818182,32.454545,6.187500,17.264706,54.187500,54.000000,26.129630,56.500000,17.376984,22.538462,...,9.323529,25.935897,73.166667,31.877778,22.271739,33.113636,53.090909,33.767241,8.934783,33.520408
5,41.189655,40.225490,3.000000,19.615854,46.797297,37.970000,29.100000,39.296875,18.568862,34.927083,...,53.442857,30.687500,81.083333,32.681034,33.946809,37.837209,8.904762,31.710526,28.891304,45.042857
6,28.869565,26.732143,15.312500,15.289256,33.109091,27.500000,10.916667,36.750000,26.895833,28.964286,...,22.382353,54.232143,41.468750,38.400000,30.787037,41.263889,18.437500,31.763158,10.475000,30.666667
7,21.250000,20.169811,14.250000,16.384615,29.053571,25.366667,8.625000,14.875000,14.809735,20.462963,...,12.833333,24.086207,32.500000,10.841463,22.197674,31.071429,14.250000,24.040000,21.666667,16.100000
8,53.125000,8.375000,26.607143,19.723529,4.833333,26.937500,26.145161,25.250000,17.115385,32.705882,...,27.500000,20.142857,11.933333,11.250000,9.250000,23.772727,18.930435,19.062500,7.250000,15.333333
9,28.500000,21.111111,18.038462,15.361963,19.625000,16.870000,9.710526,26.085366,14.217241,23.857143,...,36.333333,13.133333,40.913793,18.848837,14.354167,36.977273,25.543478,21.209877,13.361111,25.705882


In [19]:
X_train.shape

(46, 50)

In [20]:
X_val = get_local_features(df_val["text"], word_features, measures)

Nodes: 4068 - Edges: 17437
MEASURE [11.9        24.16666667 24.25       37.5        17.25               nan
 26.33333333         nan         nan 11.9                nan         nan
         nan 24.16666667         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan    

/home/mailaucq/Documents/book_classification/venv/lib/python3.8/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


Nodes: 6879 - Edges: 19481
MEASURE [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan

Nodes: 3564 - Edges: 14954
MEASURE [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan

In [21]:
X_val.shape

(16, 50)

In [22]:
classes = list(df_train['label'])  ## or 'author'
total_classes = list(set(df_train['label']))  ## or author
print("Classes: {}".format(total_classes))
print("Total classes: {}".format(len(total_classes)))
number_books = (df_train[df_train['label'] == total_classes[0]]).shape[0]
print("Total entities for each class in train: {}".format(number_books))
dict_categories = list(set(classes))
dict_categories = {cat: index for index, cat in enumerate(dict_categories)}

Classes: ['Mark Twain', 'Daniel Defoe', 'Hector Hugh', 'Arthur Conan Doyle', 'Thomas Hardy', 'Pelham Grenville', 'Charles Darwin', 'Charles Dickens', 'Bram Stoker', 'Joseph Conrad', 'Jane Austen', 'Allan Poe', 'George Eliot']
Total classes: 13
Total entities for each class in train: 3


In [23]:
y_train = [dict_categories[y] for y in df_train["label"]]

In [24]:
y_val = [dict_categories[y] for y in df_val["label"]]

In [25]:
df_train["label"].unique()

array(['Bram Stoker', 'Allan Poe', 'Hector Hugh', 'Charles Dickens',
       'Thomas Hardy', 'Arthur Conan Doyle', 'Pelham Grenville',
       'Charles Darwin', 'George Eliot', 'Daniel Defoe', 'Joseph Conrad',
       'Jane Austen', 'Mark Twain'], dtype=object)

In [26]:
df_train.isnull().values.any()

False

In [27]:
df_train.head()

,label,text,book
0,Bram Stoker,"[Produced, eagkw, Robert, Cicconetti, Online, ...",The Mystery Of The Sea
1,Allan Poe,"[Contents, DEVIL, BELFRY, LIONIZING, XING, PAR...",The Works Of Edgar Allan Poe4
2,Allan Poe,"[Contents, PURLOINED, LETTER, THOUSANDANDSECON...",The Works Of Edgar Allan Poe2
3,Hector Hugh,"[Transcribed, 1914, John, Lane, edition, David...",When William Came
4,Charles Dickens,"[Also, please, aware, spellchecking, within, d...",Barnaby Rudge


## Exploratory data analysis

In [28]:
#df_data = X_train.copy()

In [29]:
#df_data["label"] = y_train

In [30]:
#import seaborn as sns
#import matplotlib.pyplot as plt

In [31]:
#plt.close()
#sns.set_style("whitegrid")
#sns.pairplot(df_data, hue="label", height=3)
#plt.show()

In [32]:
#from sklearn.preprocessing import MinMaxScaler
#scaler = MinMaxScaler()
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(with_mean=True, with_std=True)

In [33]:
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

In [34]:
X_train

array([[ 0.24778879, -0.56576698, -0.27111944, ..., -1.08987305,
        -0.17080222, -0.52541283],
       [-1.55537017, -0.69195604, -0.60930214, ..., -0.55212541,
        -0.99720605, -2.39268991],
       [-0.60908379, -1.36699799, -0.48832622, ..., -1.95824832,
        -0.59447262, -1.54542096],
       ...,
       [-0.42263767, -0.3522936 , -0.90074415, ..., -0.068958  ,
        -0.8008677 ,  0.26916403],
       [ 2.85911715,  0.22942508, -0.10251589, ..., -0.69417295,
         0.63081956,  1.17050166],
       [-1.17044323,  0.00365387, -0.92823868, ..., -0.79955686,
        -0.72059832, -0.82503903]])

In [35]:
y_train

[8,
 11,
 11,
 2,
 7,
 4,
 3,
 5,
 6,
 12,
 1,
 5,
 3,
 4,
 11,
 11,
 2,
 9,
 1,
 7,
 10,
 12,
 9,
 4,
 2,
 3,
 0,
 3,
 2,
 0,
 7,
 12,
 0,
 6,
 6,
 10,
 12,
 8,
 8,
 10,
 1,
 8,
 9,
 10,
 1,
 5]

# Training with Bayes

In [36]:
from classifierv2 import getClassifier, getClassMetrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score

In [37]:
from sklearn.neighbors import KNeighborsClassifier

In [38]:
knn_clf = KNeighborsClassifier()
knn_clf.fit(X_train, y_train)

predicted = knn_clf.predict(X_val)
print(X_train)
print(y_train)
print(y_val)
print(predicted)
#print accuracy_score
print("accuracy : " + str(accuracy_score(y_val, predicted)))

print("micro f-measure " + str(f1_score(y_val, predicted, average='micro')))

[[ 0.24778879 -0.56576698 -0.27111944 ... -1.08987305 -0.17080222
  -0.52541283]
 [-1.55537017 -0.69195604 -0.60930214 ... -0.55212541 -0.99720605
  -2.39268991]
 [-0.60908379 -1.36699799 -0.48832622 ... -1.95824832 -0.59447262
  -1.54542096]
 ...
 [-0.42263767 -0.3522936  -0.90074415 ... -0.068958   -0.8008677
   0.26916403]
 [ 2.85911715  0.22942508 -0.10251589 ... -0.69417295  0.63081956
   1.17050166]
 [-1.17044323  0.00365387 -0.92823868 ... -0.79955686 -0.72059832
  -0.82503903]]
[8, 11, 11, 2, 7, 4, 3, 5, 6, 12, 1, 5, 3, 4, 11, 11, 2, 9, 1, 7, 10, 12, 9, 4, 2, 3, 0, 3, 2, 0, 7, 12, 0, 6, 6, 10, 12, 8, 8, 10, 1, 8, 9, 10, 1, 5]
[6, 11, 5, 3, 10, 0, 12, 9, 6, 4, 7, 8, 5, 1, 7, 2]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
accuracy : 0.0625
micro f-measure 0.0625


In [39]:
from sklearn.svm import SVC

In [40]:
svc = SVC(kernel='linear', probability=True)
svc.fit(X_train, y_train)

predicted = svc.predict(X_val)
print(X_train)
print(y_train)
print(y_val)
print(predicted)
#print accuracy_score
print("accuracy : " + str(accuracy_score(y_val, predicted)))

print("micro f-measure " + str(f1_score(y_val, predicted, average='micro')))

[[ 0.24778879 -0.56576698 -0.27111944 ... -1.08987305 -0.17080222
  -0.52541283]
 [-1.55537017 -0.69195604 -0.60930214 ... -0.55212541 -0.99720605
  -2.39268991]
 [-0.60908379 -1.36699799 -0.48832622 ... -1.95824832 -0.59447262
  -1.54542096]
 ...
 [-0.42263767 -0.3522936  -0.90074415 ... -0.068958   -0.8008677
   0.26916403]
 [ 2.85911715  0.22942508 -0.10251589 ... -0.69417295  0.63081956
   1.17050166]
 [-1.17044323  0.00365387 -0.92823868 ... -0.79955686 -0.72059832
  -0.82503903]]
[8, 11, 11, 2, 7, 4, 3, 5, 6, 12, 1, 5, 3, 4, 11, 11, 2, 9, 1, 7, 10, 12, 9, 4, 2, 3, 0, 3, 2, 0, 7, 12, 0, 6, 6, 10, 12, 8, 8, 10, 1, 8, 9, 10, 1, 5]
[6, 11, 5, 3, 10, 0, 12, 9, 6, 4, 7, 8, 5, 1, 7, 2]
[ 2 11  2  2  2  2  2  2  2  2  5  2  2  2  2  2]
accuracy : 0.125
micro f-measure 0.125
